# Portfolio IRR Curve (from fund-level mean cashflows)

This notebook computes the portfolio IRR *by quarter* using the mean fund-level cashflows from the simulation outputs.

Inputs (from sim_outputs):
- sim_fund_draw_mean.csv
- sim_fund_rep_mean.csv
- sim_fund_nav_mean.csv

Outputs:
- portfolio_irr_curve.csv (portfolio IRR by quarter)
- fund_irr_curve.csv (optional, per-fund IRR by quarter)


In [2]:

import os
from pathlib import Path
import numpy as np
import pandas as pd


In [3]:

# --- Config ---
RUN_TAG = os.environ.get('RUN_TAG', 'test_portfolio_2025Q3')
OUTPUT_NAME = "portfolio_irr_curve.csv"
WRITE_FUND_IRR = False  # set True to output per-fund IRR curve (can be large)


In [4]:
# --- Locate sim_outputs ---
def resolve_sim_outputs(run_tag: str):
    candidates = [
        Path('model_fits') / 'runs' / run_tag / 'projection' / 'sim_outputs',
        Path('model_fits') / 'model_fits' / 'runs' / run_tag / 'projection' / 'sim_outputs',
    ]
    existing = [p for p in candidates if p.exists()]
    if not existing:
        raise FileNotFoundError('sim_outputs folder not found for run_tag: ' + run_tag)
    # prefer model_fits/model_fits if present (new layout)
    pref = next((p for p in existing if 'model_fits/model_fits' in str(p)), None)
    sim_dir = pref or existing[0]
    alt_dir = next((p for p in existing if p != sim_dir), None)
    return sim_dir, alt_dir

sim_dir, alt_sim_dir = resolve_sim_outputs(RUN_TAG)
print('Using sim_outputs:', sim_dir)
if alt_sim_dir:
    print('Alt sim_outputs:', alt_sim_dir)


Using sim_outputs: model_fits/runs/test_portfolio_2025Q3/projection/sim_outputs


In [5]:
# --- IRR calculation ---
sim_dir, alt_sim_dir = resolve_sim_outputs(RUN_TAG)
draw_path = sim_dir / 'sim_fund_draw_mean.csv'
rep_path = sim_dir / 'sim_fund_rep_mean.csv'
nav_path = sim_dir / 'sim_fund_nav_mean.csv'

for p in (draw_path, rep_path, nav_path):
    if not p.exists():
        raise FileNotFoundError(f'Missing {p}')

draw = pd.read_csv(draw_path)
rep = pd.read_csv(rep_path)
nav = pd.read_csv(nav_path)

draw_q = draw.groupby('quarter_end', as_index=False)['draw_mean'].sum().rename(columns={'draw_mean':'draw'})
rep_q = rep.groupby('quarter_end', as_index=False)['rep_mean'].sum().rename(columns={'rep_mean':'rep'})
nav_q = nav.groupby('quarter_end', as_index=False)['nav_mean'].sum().rename(columns={'nav_mean':'nav'})
out = draw_q.merge(rep_q, on='quarter_end', how='outer').merge(nav_q, on='quarter_end', how='outer').sort_values('quarter_end')
out[['draw','rep','nav']] = out[['draw','rep','nav']].fillna(0.0)
out['cf'] = out['rep'] - out['draw']

def xnpv(rate, cfs, dts):
    dts = np.asarray(dts, dtype='datetime64[ns]')
    cfs = np.asarray(cfs, dtype=float)
    t0 = dts[0]
    day_counts = (dts - t0) / np.timedelta64(1, 'D')
    years = day_counts / 365.0
    return np.sum(cfs / ((1.0 + rate) ** years))

def xirr_newton(cfs, dts, guess=0.1, max_iter=80, tol=1e-7):
    dts = np.asarray(dts, dtype='datetime64[ns]')
    cfs = np.asarray(cfs, dtype=float)
    rate = float(guess)
    for _ in range(max_iter):
        f = xnpv(rate, cfs, dts)
        if not np.isfinite(f):
            return np.nan
        if abs(f) < tol:
            return rate
        eps = 1e-6
        f1 = xnpv(rate + eps, cfs, dts)
        df = (f1 - f) / eps
        if df == 0 or not np.isfinite(df):
            return np.nan
        rate_new = rate - f / df
        if rate_new <= -0.999999 or not np.isfinite(rate_new):
            return np.nan
        rate = rate_new
    return np.nan

def xirr_bisect(cfs, dts):
    # bracket search over a wide grid
    grid = [-0.9999, -0.9, -0.7, -0.5, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100]
    vals = []
    for r in grid:
        try:
            vals.append(xnpv(r, cfs, dts))
        except Exception:
            vals.append(np.nan)
    # find sign change
    for i in range(len(grid)-1):
        a,b = grid[i], grid[i+1]
        fa, fb = vals[i], vals[i+1]
        if not np.isfinite(fa) or not np.isfinite(fb):
            continue
        if fa == 0:
            return a
        if fa * fb < 0:
            # bisection
            lo, hi = a, b
            for _ in range(80):
                mid = (lo + hi) / 2.0
                fm = xnpv(mid, cfs, dts)
                if not np.isfinite(fm):
                    break
                if abs(fm) < 1e-7:
                    return mid
                if fa * fm < 0:
                    hi = mid; fb = fm
                else:
                    lo = mid; fa = fm
            return (lo + hi) / 2.0
    return np.nan

irr_vals = []
for i in range(len(out)):
    cfs = out['cf'].iloc[:i+1].to_numpy(dtype=float)
    dts = pd.to_datetime(out['quarter_end'].iloc[:i+1]).to_numpy()
    cfs_full = np.append(cfs, out['nav'].iloc[i])
    dts_full = np.append(dts, np.datetime64(out['quarter_end'].iloc[i], 'ns'))
    if not (np.any(cfs_full < 0) and np.any(cfs_full > 0)):
        irr_vals.append(np.nan)
        continue
    paid_in = -np.sum(cfs[cfs < 0])
    distributed = np.sum(cfs[cfs > 0])
    tvpi = (distributed + out['nav'].iloc[i]) / paid_in if paid_in > 0 else np.nan
    guess = 0.10 if (pd.notna(tvpi) and tvpi > 1.0) else -0.10
    irr = xirr_newton(cfs_full, dts_full, guess=guess)
    if pd.notna(tvpi) and (0.98 <= tvpi <= 1.02) and not np.isfinite(irr):
        irr2 = xirr_newton(cfs_full, dts_full, guess=-guess)
        irr = irr2 if np.isfinite(irr2) else np.nan
    if not np.isfinite(irr):
        irr = xirr_bisect(cfs_full, dts_full)
    irr_vals.append(irr)

out['portfolio_irr'] = irr_vals
out_path = sim_dir / OUTPUT_NAME
out_path.unlink(missing_ok=True)
out.to_csv(out_path, index=False)
print('Wrote:', out_path)


Wrote: model_fits/runs/test_portfolio_2025Q3/projection/sim_outputs/portfolio_irr_curve.csv
